In [47]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
df = pd.read_csv('../../flight-delay/extracted_data/mysql/db/2003.csv')
df.head()

,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,Reporting_Airline,DOT_ID_Reporting_Airline,IATA_CODE_Reporting_Airline,Tail_Number,Flight_Number_Reporting_Airline,OriginAirportID,OriginAirportSeqID,OriginCityMarketID,Origin,OriginCityName,OriginState,OriginStateFips,OriginStateName,OriginWac,DestAirportID,DestAirportSeqID,DestCityMarketID,Dest,DestCityName,DestState,DestStateFips,DestStateName,DestWac,CRSDepTime,DepTime,DepDelay,DepDelayMinutes,DepDel15,DepartureDelayGroups,DepTimeBlk,TaxiOut,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrTime,ArrDelay,ArrDelayMinutes,ArrDel15,ArrivalDelayGroups,ArrTimeBlk,Cancelled,CancellationCode,Diverted,CRSElapsedTime,ActualElapsedTime,AirTime,Flights,Distance,DistanceGroup,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,Id
0,2020,1,3,9,1,2020-03-09,9E,20363,9E,N669CA,4677,14307,1430705,30721,PVD,"Providence, RI",RI,44,Rhode Island,15,11433,1143302,31295,DTW,"Detroit, MI",MI,26,Michigan,43,1650,1646.0,-4.0,0.0,0.0,-1.0,1600-1659,18.0,1704.0,1846.0,7.0,1909,1853.0,-16.0,0.0,0.0,-2.0,1900-1959,0.0,NaN,0.0,139.0,127.0,102.0,1.0,615.0,3,NaN,NaN,NaN,NaN,NaN,1181615
1,2020,1,3,7,6,2020-03-07,9E,20363,9E,N980EV,4678,15380,1538005,35380,TVC,"Traverse City, MI",MI,26,Michigan,43,11433,1143302,31295,DTW,"Detroit, MI",MI,26,Michigan,43,1759,1743.0,-16.0,0.0,0.0,-2.0,1700-1759,11.0,1754.0,1829.0,10.0,1921,1839.0,-42.0,0.0,0.0,-2.0,1900-1959,0.0,NaN,0.0,82.0,56.0,35.0,1.0,207.0,1,NaN,NaN,NaN,NaN,NaN,1181617
2,2020,1,3,7,6,2020-03-07,9E,20363,9E,N836AY,4680,11433,1143302,31295,DTW,"Detroit, MI",MI,26,Michigan,43,10469,1046902,30469,AZO,"Kalamazoo, MI",MI,26,Michigan,43,1220,1216.0,-4.0,0.0,0.0,-1.0,1200-1259,15.0,1231.0,1255.0,5.0,1319,1300.0,-19.0,0.0,0.0,-2.0,1300-1359,0.0,NaN,0.0,59.0,44.0,24.0,1.0,113.0,1,NaN,NaN,NaN,NaN,NaN,1181619
3,2020,1,3,22,7,2020-03-22,9E,20363,9E,N8896A,4682,10469,1046902,30469,AZO,"Kalamazoo, MI",MI,26,Michigan,43,11433,1143302,31295,DTW,"Detroit, MI",MI,26,Michigan,43,957,957.0,0.0,0.0,0.0,0.0,0900-0959,9.0,1006.0,1032.0,11.0,1101,1043.0,-18.0,0.0,0.0,-2.0,1100-1159,0.0,NaN,0.0,64.0,46.0,26.0,1.0,113.0,1,NaN,NaN,NaN,NaN,NaN,1181621
4,2020,1,3,22,7,2020-03-22,9E,20363,9E,N8896A,4682,11433,1143302,31295,DTW,"Detroit, MI",MI,26,Michigan,43,10469,1046902,30469,AZO,"Kalamazoo, MI",MI,26,Michigan,43,835,835.0,0.0,0.0,0.0,0.0,0800-0859,17.0,852.0,918.0,3.0,932,921.0,-11.0,0.0,0.0,-1.0,0900-0959,0.0,NaN,0.0,57.0,46.0,26.0,1.0,113.0,1,NaN,NaN,NaN,NaN,NaN,1181623


In [52]:
# import libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

# initialize spark session
spark = SparkSession \
        .builder \
        .master("local[*]") \
        .appName("StagingToWarehouse") \
        .config("spark.executor.memory", "2g") \
        .config("spark.executor.cores", '2') \
        .config("spark.driver.memory", '2g') \
        .config("spark.driver.cores", '2') \
        .config("spark.cassandra.connection.host", "localhost") \
        .config("spark.cassandra.connection.port", "9042") \
        .config("spark.cassandra.auth.username", "root") \
        .config("spark.cassandra.auth.password", "admin") \
        .config("spark.jars", "../../tools/spark-jars/spark-3.3-bigquery-0.31.1.jar") \
        .config("spark.hadoop.google.cloud.auth.service.account.enable", "true") \
        .config("spark.hadoop.google.cloud.auth.service.account.json.keyfile", "../../tools/bigquery/key_file.json") \
        .getOrCreate()

In [53]:
# get max id of fact_records table
fact_records_max_id = spark.read \
                        .format("bigquery") \
                        .options(parentProject="test-373705") \
                        .options(table="flight_delay.fact_records") \
                        .load().select("id") \
                        .agg(max("id")) \
                        .collect()[0][0]
# ensure max id have a value
if fact_records_max_id is None:
    fact_records_max_id=-1

# extract data from mysql table
mysql = spark.read \
            .format("org.apache.spark.sql.cassandra") \
            .options(keyspace="flight_delay") \
            .options(table="mysql") \
            .load() \
            .filter(col("id")>fact_records_max_id)
# extract data from mongodb table
mongodb = spark.read \
            .format("org.apache.spark.sql.cassandra") \
            .options(keyspace="flight_delay") \
            .options(table="mongodb") \
            .load() \
            .filter(col("id")>fact_records_max_id)
# integrate data, rename columns, and change datatype
source = mysql.union(mongodb).select(
    col("id").alias("id"),
    col("actualelapsedtime").cast("bigint").alias("actual_elapsed_time"),
    col("airtime").cast("bigint").alias("air_time"),
    col("arrdel15").cast("bigint").alias("arr_delay_15"),
    col("arrdelay").cast("bigint").alias("arr_delay"),
    col("arrdelayminutes").cast("bigint").alias("arr_delay_minutes"),
    col("arrivaldelaygroups").cast("bigint").alias("arr_delay_groups"),
    col("arrtime").cast("bigint").alias("arr_time"),
    col("arrtimeblk").alias("arr_time_block"),
    col("cancellationcode").alias("cancellation_code"),
    col("cancelled").cast("bigint"),
    col("carrierdelay").cast("bigint").alias("carrier_delay"),
    col("crsarrtime").alias("crs_arr_time"),
    col("crsdeptime").alias("crs_dep_time"),
    col("crselapsedtime").cast("bigint").alias("crs_elapsed_time"),
    col("dayofmonth").alias("day_of_month"),
    col("dayofweek").alias("day_of_week"),
    col("departuredelaygroups").cast("bigint").alias("dep_delay_groups"),
    col("depdel15").cast("bigint").alias("dep_delay_15"),
    col("depdelay").cast("bigint").alias("dep_delay"),
    col("depdelayminutes").cast("bigint").alias("dep_delay_minutes"),
    col("deptime").cast("bigint").alias("dep_time"),
    col("deptimeblk").alias("dep_time_block"),
    col("dest"),
    col("destairportid").alias("dest_airport_id"),
    col("destairportseqid").alias("dest_airport_seq_id"),
    col("destcitymarketid").alias("dest_city_market_id"),
    col("destcityname").alias("dest_city_name"),
    col("deststate").alias("dest_state"),
    col("deststatefips").alias("dest_state_fips"),
    col("deststatename").alias("dest_state_name"),
    col("destwac").alias("dest_wac"),
    col("distance").cast("bigint"),
    col("distancegroup").alias("distance_group"),
    col("diverted").cast("bigint").alias("diverted"),
    col("dot_id_reporting_airline"),
    col("flight_number_reporting_airline"),
    col("flightdate").cast("date").alias("flight_date"),
    col("flights").cast("bigint"),
    col("iata_code_reporting_airline"),
    col("lateaircraftdelay").cast("bigint").alias("late_aircraft_delay"),
    col("month"),
    col("nasdelay").cast("bigint").alias("nas_delay"),
    col("origin"),
    col("originairportid").alias("origin_airport_id"),
    col("originairportseqid").alias("origin_airport_seq_id"),
    col("origincitymarketid").alias("origin_city_market_id"),
    col("origincityname").alias("origin_city_name"),
    col("originstate").alias("origin_state"),
    col("originstatefips").alias("origin_state_fips"),
    col("originstatename").alias("origin_state_name"),
    col("originwac").alias("origin_wac"),
    col("quarter"),
    col("reporting_airline"),
    col("securitydelay").alias("security_delay"),
    col("tail_number"),
    col("taxiin").cast("bigint").alias("taxi_in"),
    col("taxiout").cast("bigint").alias("taxi_out"),
    col("weatherdelay").cast("bigint").alias("weather_delay"),
    col("wheelsoff").cast("bigint").alias("wheels_on"),
    col("wheelson").cast("bigint").alias("wheels_off"),
    col("year")
)

23/07/02 21:57:11 WARN ControlConnection: [s0] Error connecting to Node(endPoint=localhost/0:0:0:0:0:0:0:1:9042, hostId=null, hashCode=5abcda5f), trying next node (ConnectionInitException: [s0|control|connecting...] Protocol initialization request, step 1 (OPTIONS): failed to send request (com.datastax.oss.driver.shaded.netty.channel.StacklessClosedChannelException))


In [54]:
source.printSchema()

root
 |-- id: long (nullable = false)
 |-- actual_elapsed_time: long (nullable = true)
 |-- air_time: long (nullable = true)
 |-- arr_delay_15: long (nullable = true)
 |-- arr_delay: long (nullable = true)
 |-- arr_delay_minutes: long (nullable = true)
 |-- arr_delay_groups: long (nullable = true)
 |-- arr_time: long (nullable = true)
 |-- arr_time_block: string (nullable = true)
 |-- cancellation_code: string (nullable = true)
 |-- cancelled: long (nullable = true)
 |-- carrier_delay: long (nullable = true)
 |-- crs_arr_time: long (nullable = true)
 |-- crs_dep_time: long (nullable = true)
 |-- crs_elapsed_time: long (nullable = true)
 |-- day_of_month: long (nullable = true)
 |-- day_of_week: long (nullable = true)
 |-- dep_delay_groups: long (nullable = true)
 |-- dep_delay_15: long (nullable = true)
 |-- dep_delay: long (nullable = true)
 |-- dep_delay_minutes: long (nullable = true)
 |-- dep_time: long (nullable = true)
 |-- dep_time_block: string (nullable = true)
 |-- dest: stri

In [65]:
# get id, flight_date from dim_date table
current_dim_date = spark.read \
                    .format("bigquery") \
                    .options(parentProject="test-373705") \
                    .options(table="flight_delay.dim_date") \
                    .load().select("id", "flight_date")
# get new data from source dataframe
new_dim_date = source.select("year", "quarter", "month", "day_of_month", "day_of_week", "flight_date").distinct()
# mapping
joined_dim_date = new_dim_date.join(current_dim_date, on="flight_date", how="full")
# fullfill null id
dim_date = joined_dim_date.filter(col("id").isNull()) \
                    .orderBy(col("flight_date").asc()) \
                    .withColumn("id", monotonically_increasing_id())
# add date_id to source dataframe
date_id_source = source.join(dim_date.selectExpr("flight_date", "id as date_id"), how="left")
# get max id of dim_date table
max_date_id = current_dim_date.agg(max("id")).collect()[0][0]
# ensure max id have a value
if max_date_id is None:
    max_date_id=-1
# drop old data
to_append_dim_date = dim_date.filter(col("id")>max_date_id)

In [ ]:
# append new data to dim_date table
to_append_dim_date.write \
            .format("bigquery") \
            .options(parentProject="test-373705") \
            .options(table="flight_delay.dim_date") \
            .option("writeMethod", "direct") \
            .mode("append") \
            .save()

In [67]:
# export new data to csv
to_append_dim_date.select("id", "year", "quarter", "month", "day_of_month", "day_of_week", "flight_date") \
            .repartition(1) \
            .write \
            .format("csv") \
            .options(header="True") \
            .save("../../flight-delay/bigquery/dim_date")

23/07/02 22:31:43 WARN CqlRequestHandler: Query '[2 values] SELECT "id", "dayofmonth", "dayofweek", "flightdate", "month", "quarter", "year" FROM flight_delay.mysql WHERE token("id") > ? AND token("id") <= ?   ALLOW FILTERING ["partition key token"=4135204860069402604, "partition key token"=5226501970761614863]' generated server side warning(s): Read 1000 live rows and 6770 tombstone cells for query SELECT dayofmonth, dayofweek, flightdate, month, quarter, year FROM flight_delay.mysql WHERE token(id) > 4135204860069402604 AND token(id) <= 5226501970761614863 LIMIT 1000 ALLOW FILTERING; token 4153932036186976167 (see tombstone_warn_threshold)
23/07/02 22:31:43 WARN CqlRequestHandler: Query '[2 values] SELECT "id", "dayofmonth", "dayofweek", "flightdate", "month", "quarter", "year" FROM flight_delay.mysql WHERE token("id") > ? AND token("id") <= ?   ALLOW FILTERING ["partition key token"=-2926644641716178823, "partition key token"=-1336640089202280185]' generated server side warning(s): 

23/07/02 22:31:43 WARN CqlRequestHandler: Query '[2 values] SELECT "id", "dayofmonth", "dayofweek", "flightdate", "month", "quarter", "year" FROM flight_delay.mysql WHERE token("id") > ? AND token("id") <= ?   ALLOW FILTERING ["partition key token"=-6216901697922907952, "partition key token"=-5137528890475306295]' generated server side warning(s): Read 1000 live rows and 7007 tombstone cells for query SELECT dayofmonth, dayofweek, flightdate, month, quarter, year FROM flight_delay.mysql WHERE token(id) > token(127467) AND token(id) <= -5137528890475306295 LIMIT 1000 ALLOW FILTERING; token -6142222145296081502 (see tombstone_warn_threshold)
23/07/02 22:31:43 WARN CqlRequestHandler: Query '[2 values] SELECT "id", "dayofmonth", "dayofweek", "flightdate", "month", "quarter", "year" FROM flight_delay.mysql WHERE token("id") > ? AND token("id") <= ?   ALLOW FILTERING ["partition key token"=-8028573271859381133, "partition key token"=-6952817390633801593]' generated server side warning(s): Re

23/07/02 22:31:43 WARN CqlRequestHandler: Query '[2 values] SELECT "id", "dayofmonth", "dayofweek", "flightdate", "month", "quarter", "year" FROM flight_delay.mysql WHERE token("id") > ? AND token("id") <= ?   ALLOW FILTERING ["partition key token"=4135204860069402604, "partition key token"=5226501970761614863]' generated server side warning(s): Read 1000 live rows and 6947 tombstone cells for query SELECT dayofmonth, dayofweek, flightdate, month, quarter, year FROM flight_delay.mysql WHERE token(id) > token(1106833) AND token(id) <= 5226501970761614863 LIMIT 1000 ALLOW FILTERING; token 4359841927791944902 (see tombstone_warn_threshold)
23/07/02 22:31:43 WARN CqlRequestHandler: Query '[2 values] SELECT "id", "dayofmonth", "dayofweek", "flightdate", "month", "quarter", "year" FROM flight_delay.mysql WHERE token("id") > ? AND token("id") <= ?   ALLOW FILTERING ["partition key token"=1260528659010877187, "partition key token"=2149845809422670751]' generated server side warning(s): Read 10

23/07/02 22:31:43 WARN CqlRequestHandler: Query '[2 values] SELECT "id", "dayofmonth", "dayofweek", "flightdate", "month", "quarter", "year" FROM flight_delay.mysql WHERE token("id") > ? AND token("id") <= ?   ALLOW FILTERING ["partition key token"=-5137528890475306295, "partition key token"=-3894008884794714321]' generated server side warning(s): Read 1000 live rows and 6933 tombstone cells for query SELECT dayofmonth, dayofweek, flightdate, month, quarter, year FROM flight_delay.mysql WHERE token(id) > token(191479) AND token(id) <= -3894008884794714321 LIMIT 1000 ALLOW FILTERING; token -4805987062470358728 (see tombstone_warn_threshold)
23/07/02 22:31:43 WARN CqlRequestHandler: Query '[2 values] SELECT "id", "dayofmonth", "dayofweek", "flightdate", "month", "quarter", "year" FROM flight_delay.mysql WHERE token("id") > ? AND token("id") <= ?   ALLOW FILTERING ["partition key token"=-2926644641716178823, "partition key token"=-1336640089202280185]' generated server side warning(s): Re

23/07/02 22:31:43 WARN CqlRequestHandler: Query '[2 values] SELECT "id", "dayofmonth", "dayofweek", "flightdate", "month", "quarter", "year" FROM flight_delay.mysql WHERE token("id") > ? AND token("id") <= ?   ALLOW FILTERING ["partition key token"=-5137528890475306295, "partition key token"=-3894008884794714321]' generated server side warning(s): Read 1000 live rows and 6958 tombstone cells for query SELECT dayofmonth, dayofweek, flightdate, month, quarter, year FROM flight_delay.mysql WHERE token(id) > token(1713833) AND token(id) <= -3894008884794714321 LIMIT 1000 ALLOW FILTERING; token -4657750880601375641 (see tombstone_warn_threshold)
23/07/02 22:31:43 WARN CqlRequestHandler: Query '[2 values] SELECT "id", "dayofmonth", "dayofweek", "flightdate", "month", "quarter", "year" FROM flight_delay.mysql WHERE token("id") > ? AND token("id") <= ?   ALLOW FILTERING ["partition key token"=-3894008884794714321, "partition key token"=-2926644641716178823]' generated server side warning(s): R

...

23/07/02 22:31:46 WARN CqlRequestHandler: Query '[2 values] SELECT "id", "dayofmonth", "dayofweek", "flightdate", "month", "quarter", "year" FROM flight_delay.mysql WHERE token("id") > ? AND token("id") <= ?   ALLOW FILTERING ["partition key token"=6070878618252776621, "partition key token"=7511146848155359859]' generated server side warning(s): Read 1000 live rows and 6949 tombstone cells for query SELECT dayofmonth, dayofweek, flightdate, month, quarter, year FROM flight_delay.mysql WHERE token(id) > token(861349) AND token(id) <= 7511146848155359859 LIMIT 1000 ALLOW FILTERING; token 7490229769169924256 (see tombstone_warn_threshold)
23/07/02 22:31:46 WARN CqlRequestHandler: Query '[2 values] SELECT "id", "dayofmonth", "dayofweek", "flightdate", "month", "quarter", "year" FROM flight_delay.mysql WHERE token("id") > ? AND token("id") <= ?   ALLOW FILTERING ["partition key token"=282383206494763111, "partition key token"=1260528659010877187]' generated server side warning(s): Read 1000

23/07/02 22:31:47 WARN CqlRequestHandler: Query '[2 values] SELECT "id", "dayofmonth", "dayofweek", "flightdate", "month", "quarter", "year" FROM flight_delay.mysql WHERE token("id") > ? AND token("id") <= ?   ALLOW FILTERING ["partition key token"=-1336640089202280185, "partition key token"=282383206494763111]' generated server side warning(s): Read 1000 live rows and 7056 tombstone cells for query SELECT dayofmonth, dayofweek, flightdate, month, quarter, year FROM flight_delay.mysql WHERE token(id) > token(1538591) AND token(id) <= 282383206494763111 LIMIT 1000 ALLOW FILTERING; token -615997881548171108 (see tombstone_warn_threshold)
23/07/02 22:31:47 WARN CqlRequestHandler: Query '[2 values] SELECT "id", "dayofmonth", "dayofweek", "flightdate", "month", "quarter", "year" FROM flight_delay.mysql WHERE token("id") > ? AND token("id") <= ?   ALLOW FILTERING ["partition key token"=282383206494763111, "partition key token"=1260528659010877187]' generated server side warning(s): Read 1000

23/07/02 22:31:47 WARN CqlRequestHandler: Query '[2 values] SELECT "id", "dayofmonth", "dayofweek", "flightdate", "month", "quarter", "year" FROM flight_delay.mysql WHERE token("id") > ? AND token("id") <= ?   ALLOW FILTERING ["partition key token"=-1336640089202280185, "partition key token"=282383206494763111]' generated server side warning(s): Read 1000 live rows and 6600 tombstone cells for query SELECT dayofmonth, dayofweek, flightdate, month, quarter, year FROM flight_delay.mysql WHERE token(id) > token(56173) AND token(id) <= 282383206494763111 LIMIT 1000 ALLOW FILTERING; token -287326694072285016 (see tombstone_warn_threshold)
23/07/02 22:31:47 WARN CqlRequestHandler: Query '[2 values] SELECT "id", "dayofmonth", "dayofweek", "flightdate", "month", "quarter", "year" FROM flight_delay.mysql WHERE token("id") > ? AND token("id") <= ?   ALLOW FILTERING ["partition key token"=-1336640089202280185, "partition key token"=282383206494763111]' generated server side warning(s): Read 1000 

23/07/02 22:31:47 WARN CqlRequestHandler: Query '[2 values] SELECT "id", "dayofmonth", "dayofweek", "flightdate", "month", "quarter", "year" FROM flight_delay.mysql WHERE token("id") > ? AND token("id") <= ?   ALLOW FILTERING ["partition key token"=-1336640089202280185, "partition key token"=282383206494763111]' generated server side warning(s): Read 1000 live rows and 6818 tombstone cells for query SELECT dayofmonth, dayofweek, flightdate, month, quarter, year FROM flight_delay.mysql WHERE token(id) > token(395567) AND token(id) <= 282383206494763111 LIMIT 1000 ALLOW FILTERING; token -138078714560767492 (see tombstone_warn_threshold)
23/07/02 22:31:47 WARN CqlRequestHandler: Query '[2 values] SELECT "id", "dayofmonth", "dayofweek", "flightdate", "month", "quarter", "year" FROM flight_delay.mysql WHERE token("id") > ? AND token("id") <= ?   ALLOW FILTERING ["partition key token"=-1336640089202280185, "partition key token"=282383206494763111]' generated server side warning(s): Read 1000

23/07/02 22:31:48 WARN CqlRequestHandler: Query '[2 values] SELECT "id", "dayofmonth", "dayofweek", "flightdate", "month", "quarter", "year" FROM flight_delay.mysql WHERE token("id") > ? AND token("id") <= ?   ALLOW FILTERING ["partition key token"=-1336640089202280185, "partition key token"=282383206494763111]' generated server side warning(s): Read 1000 live rows and 6836 tombstone cells for query SELECT dayofmonth, dayofweek, flightdate, month, quarter, year FROM flight_delay.mysql WHERE token(id) > token(1593177) AND token(id) <= 282383206494763111 LIMIT 1000 ALLOW FILTERING; token 172220191776503593 (see tombstone_warn_threshold)
23/07/02 22:31:48 WARN CqlRequestHandler: Query '[2 values] SELECT "id", "dayofmonth", "dayofweek", "flightdate", "month", "quarter", "year" FROM flight_delay.mysql WHERE token("id") > ? AND token("id") <= ?   ALLOW FILTERING ["partition key token"=-1336640089202280185, "partition key token"=282383206494763111]' generated server side warning(s): Read 1000

In [56]:
# get id, flight_date from dim_airline table
current_dim_airline = spark.read \
                    .format("bigquery") \
                    .options(parentProject="test-373705") \
                    .options(table="flight_delay.dim_airline") \
                    .load().select("id", "reporting_airline", "tail_number", "flight_number_reporting_airline")
# get new data from source dataframe
new_dim_airline = date_id_source.select("reporting_airline", "dot_id_reporting_airline",
                                            "iata_code_reporting_airline", "tail_number",
                                            "flight_number_reporting_airline").distinct()
# mapping
joined_dim_airline = new_dim_airline.join(current_dim_airline,
                                       on=["reporting_airline", "tail_number", "flight_number_reporting_airline"],
                                       how="full")
# fullfill null id
dim_airline = joined_dim_airline.filter(col("id").isNull()) \
                    .orderBy(col("reporting_airline").asc(),
                             col("tail_number").asc(),
                             col("flight_number_reporting_airline").asc()) \
                    .withColumn("id", monotonically_increasing_id())
# add airline_id to source dataframe
airline_id_source = date_id_source.join(dim_airline.selectExpr("reporting_airline", "tail_number",
                                                               "flight_number_reporting_airline", "id as airline_id"), how="left")
# get max id of dim_airline table
max_airline_id = current_dim_airline.agg(max("id")).collect()[0][0]
# ensure max id have a value
if max_airline_id is None:
    max_airline_id=-1
# drop old data
to_append_dim_airline = dim_airline.filter(col("id")>max_airline_id)

In [ ]:
# append new data to dim_airline table
to_append_dim_airline.write \
            .format("bigquery") \
            .options(parentProject="test-373705") \
            .options(table="flight_delay.dim_airline") \
            .option("writeMethod", "direct") \
            .mode("append") \
            .save()

In [64]:
# export new data to csv
to_append_dim_airline.select("id", "reporting_airline", "dot_id_reporting_airline", "iata_code_reporting_airline",
                             "tail_number", "flight_number_reporting_airline") \
            .repartition(1) \
            .write \
            .format("csv") \
            .options(header="True") \
            .save("../../flight-delay/bigquery/dim_airline")

23/07/02 22:27:32 WARN CqlRequestHandler: Query '[2 values] SELECT "id", "dot_id_reporting_airline", "flight_number_reporting_airline", "iata_code_reporting_airline", "reporting_airline", "tail_number" FROM flight_delay.mysql WHERE token("id") > ? AND token("id") <= ?   ALLOW FILTERING ["partition key token"=4135204860069402604, "partition key token"=5226501970761614863]' generated server side warning(s): Read 1000 live rows and 6770 tombstone cells for query SELECT dot_id_reporting_airline, flight_number_reporting_airline, iata_code_reporting_airline, reporting_airline, tail_number FROM flight_delay.mysql WHERE token(id) > 4135204860069402604 AND token(id) <= 5226501970761614863 LIMIT 1000 ALLOW FILTERING; token 4153932036186976167 (see tombstone_warn_threshold)
23/07/02 22:27:32 WARN CqlRequestHandler: Query '[2 values] SELECT "id", "dot_id_reporting_airline", "flight_number_reporting_airline", "iata_code_reporting_airline", "reporting_airline", "tail_number" FROM flight_delay.mysql 

23/07/02 22:27:32 WARN CqlRequestHandler: Query '[2 values] SELECT "id", "dot_id_reporting_airline", "flight_number_reporting_airline", "iata_code_reporting_airline", "reporting_airline", "tail_number" FROM flight_delay.mysql WHERE token("id") > ? AND token("id") <= ?   ALLOW FILTERING ["partition key token"=6070878618252776621, "partition key token"=7511146848155359859]' generated server side warning(s): Read 1000 live rows and 6951 tombstone cells for query SELECT dot_id_reporting_airline, flight_number_reporting_airline, iata_code_reporting_airline, reporting_airline, tail_number FROM flight_delay.mysql WHERE token(id) > token(1818625) AND token(id) <= 7511146848155359859 LIMIT 1000 ALLOW FILTERING; token 6164758765166269791 (see tombstone_warn_threshold)
23/07/02 22:27:32 WARN CqlRequestHandler: Query '[2 values] SELECT "id", "dot_id_reporting_airline", "flight_number_reporting_airline", "iata_code_reporting_airline", "reporting_airline", "tail_number" FROM flight_delay.mysql WHERE

23/07/02 22:27:32 WARN CqlRequestHandler: Query '[2 values] SELECT "id", "dot_id_reporting_airline", "flight_number_reporting_airline", "iata_code_reporting_airline", "reporting_airline", "tail_number" FROM flight_delay.mysql WHERE token("id") > ? AND token("id") <= ?   ALLOW FILTERING ["partition key token"=6070878618252776621, "partition key token"=7511146848155359859]' generated server side warning(s): Read 1000 live rows and 6828 tombstone cells for query SELECT dot_id_reporting_airline, flight_number_reporting_airline, iata_code_reporting_airline, reporting_airline, tail_number FROM flight_delay.mysql WHERE token(id) > token(1532973) AND token(id) <= 7511146848155359859 LIMIT 1000 ALLOW FILTERING; token 6276922547865399837 (see tombstone_warn_threshold)
23/07/02 22:27:32 WARN CqlRequestHandler: Query '[2 values] SELECT "id", "dot_id_reporting_airline", "flight_number_reporting_airline", "iata_code_reporting_airline", "reporting_airline", "tail_number" FROM flight_delay.mysql WHERE

23/07/02 22:27:32 WARN CqlRequestHandler: Query '[2 values] SELECT "id", "dot_id_reporting_airline", "flight_number_reporting_airline", "iata_code_reporting_airline", "reporting_airline", "tail_number" FROM flight_delay.mysql WHERE token("id") > ? AND token("id") <= ?   ALLOW FILTERING ["partition key token"=4135204860069402604, "partition key token"=5226501970761614863]' generated server side warning(s): Read 1000 live rows and 7143 tombstone cells for query SELECT dot_id_reporting_airline, flight_number_reporting_airline, iata_code_reporting_airline, reporting_airline, tail_number FROM flight_delay.mysql WHERE token(id) > token(1052769) AND token(id) <= 5226501970761614863 LIMIT 1000 ALLOW FILTERING; token 4490581225597472761 (see tombstone_warn_threshold)
23/07/02 22:27:32 WARN CqlRequestHandler: Query '[2 values] SELECT "id", "dot_id_reporting_airline", "flight_number_reporting_airline", "iata_code_reporting_airline", "reporting_airline", "tail_number" FROM flight_delay.mysql WHERE

23/07/02 22:27:33 WARN CqlRequestHandler: Query '[2 values] SELECT "id", "dot_id_reporting_airline", "flight_number_reporting_airline", "iata_code_reporting_airline", "reporting_airline", "tail_number" FROM flight_delay.mysql WHERE token("id") > ? AND token("id") <= ?   ALLOW FILTERING ["partition key token"=-8846245204471731928, "partition key token"=-8028573271859381133]' generated server side warning(s): Read 1000 live rows and 6847 tombstone cells for query SELECT dot_id_reporting_airline, flight_number_reporting_airline, iata_code_reporting_airline, reporting_airline, tail_number FROM flight_delay.mysql WHERE token(id) > token(481745) AND token(id) <= -8028573271859381133 LIMIT 1000 ALLOW FILTERING; token -8425968531902120264 (see tombstone_warn_threshold)
23/07/02 22:27:33 WARN CqlRequestHandler: Query '[2 values] SELECT "id", "dot_id_reporting_airline", "flight_number_reporting_airline", "iata_code_reporting_airline", "reporting_airline", "tail_number" FROM flight_delay.mysql WH

...

23/07/02 22:27:37 WARN CqlRequestHandler: Query '[2 values] SELECT "id", "dot_id_reporting_airline", "flight_number_reporting_airline", "iata_code_reporting_airline", "reporting_airline", "tail_number" FROM flight_delay.mysql WHERE token("id") > ? AND token("id") <= ?   ALLOW FILTERING ["partition key token"=-1336640089202280185, "partition key token"=282383206494763111]' generated server side warning(s): Read 1000 live rows and 6997 tombstone cells for query SELECT dot_id_reporting_airline, flight_number_reporting_airline, iata_code_reporting_airline, reporting_airline, tail_number FROM flight_delay.mysql WHERE token(id) > token(1563341) AND token(id) <= 282383206494763111 LIMIT 1000 ALLOW FILTERING; token 119050955680665628 (see tombstone_warn_threshold)
23/07/02 22:27:37 WARN CqlRequestHandler: Query '[2 values] SELECT "id", "dot_id_reporting_airline", "flight_number_reporting_airline", "iata_code_reporting_airline", "reporting_airline", "tail_number" FROM flight_delay.mysql WHERE t

23/07/02 22:27:37 WARN CqlRequestHandler: Query '[2 values] SELECT "id", "dot_id_reporting_airline", "flight_number_reporting_airline", "iata_code_reporting_airline", "reporting_airline", "tail_number" FROM flight_delay.mysql WHERE token("id") > ? AND token("id") <= ?   ALLOW FILTERING ["partition key token"=-1336640089202280185, "partition key token"=282383206494763111]' generated server side warning(s): Read 1000 live rows and 7019 tombstone cells for query SELECT dot_id_reporting_airline, flight_number_reporting_airline, iata_code_reporting_airline, reporting_airline, tail_number FROM flight_delay.mysql WHERE token(id) > token(410191) AND token(id) <= 282383206494763111 LIMIT 1000 ALLOW FILTERING; token 245494852083871788 (see tombstone_warn_threshold)
23/07/02 22:27:37 WARN CqlRequestHandler: Query '[2 values] SELECT "id", "dot_id_reporting_airline", "flight_number_reporting_airline", "iata_code_reporting_airline", "reporting_airline", "tail_number" FROM flight_delay.mysql WHERE to

23/07/02 22:27:38 WARN CqlRequestHandler: Query '[2 values] SELECT "id", "dot_id_reporting_airline", "flight_number_reporting_airline", "iata_code_reporting_airline", "reporting_airline", "tail_number" FROM flight_delay.mysql WHERE token("id") > ? AND token("id") <= ?   ALLOW FILTERING ["partition key token"=-2926644641716178823, "partition key token"=-1336640089202280185]' generated server side warning(s): Read 1000 live rows and 7005 tombstone cells for query SELECT dot_id_reporting_airline, flight_number_reporting_airline, iata_code_reporting_airline, reporting_airline, tail_number FROM flight_delay.mysql WHERE token(id) > token(883801) AND token(id) <= -1336640089202280185 LIMIT 1000 ALLOW FILTERING; token -1961030616146839827 (see tombstone_warn_threshold)
23/07/02 22:27:38 WARN CqlRequestHandler: Query '[2 values] SELECT "id", "dot_id_reporting_airline", "flight_number_reporting_airline", "iata_code_reporting_airline", "reporting_airline", "tail_number" FROM flight_delay.mysql WH

23/07/02 22:27:38 WARN CqlRequestHandler: Query '[2 values] SELECT "id", "dot_id_reporting_airline", "flight_number_reporting_airline", "iata_code_reporting_airline", "reporting_airline", "tail_number" FROM flight_delay.mysql WHERE token("id") > ? AND token("id") <= ?   ALLOW FILTERING ["partition key token"=-2926644641716178823, "partition key token"=-1336640089202280185]' generated server side warning(s): Read 1000 live rows and 6727 tombstone cells for query SELECT dot_id_reporting_airline, flight_number_reporting_airline, iata_code_reporting_airline, reporting_airline, tail_number FROM flight_delay.mysql WHERE token(id) > token(287825) AND token(id) <= -1336640089202280185 LIMIT 1000 ALLOW FILTERING; token -1830455290658122154 (see tombstone_warn_threshold)
23/07/02 22:27:38 WARN CqlRequestHandler: Query '[2 values] SELECT "id", "dot_id_reporting_airline", "flight_number_reporting_airline", "iata_code_reporting_airline", "reporting_airline", "tail_number" FROM flight_delay.mysql WH

23/07/02 22:27:38 WARN CqlRequestHandler: Query '[2 values] SELECT "id", "dot_id_reporting_airline", "flight_number_reporting_airline", "iata_code_reporting_airline", "reporting_airline", "tail_number" FROM flight_delay.mysql WHERE token("id") > ? AND token("id") <= ?   ALLOW FILTERING ["partition key token"=-2926644641716178823, "partition key token"=-1336640089202280185]' generated server side warning(s): Read 1000 live rows and 6670 tombstone cells for query SELECT dot_id_reporting_airline, flight_number_reporting_airline, iata_code_reporting_airline, reporting_airline, tail_number FROM flight_delay.mysql WHERE token(id) > token(104917) AND token(id) <= -1336640089202280185 LIMIT 1000 ALLOW FILTERING; token -1574064709230381675 (see tombstone_warn_threshold)
23/07/02 22:27:38 WARN CqlRequestHandler: Query '[2 values] SELECT "id", "dot_id_reporting_airline", "flight_number_reporting_airline", "iata_code_reporting_airline", "reporting_airline", "tail_number" FROM flight_delay.mysql WH

In [1]:
spark.stop()